# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [2]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset.csv


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [5]:
df = pd.read_csv('dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [16]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [17]:
df["PreviousPrice"] = 10.0
df["NewPrice"] = 0.0

def adaptive_price(prev_price, occupancy, capacity, alpha=2.0, k=2.0):
    if capacity == 0:
        return prev_price
    occ_rate = occupancy / capacity
    return round(prev_price + alpha * (occ_rate ** k), 2)

results = []

for lot in df["SystemCodeNumber"].unique():
    temp = df[df["SystemCodeNumber"] == lot].copy()
    temp = temp.sort_values("Timestamp").reset_index(drop=True)

    temp["PreviousPrice"] = 10.0
    temp["NewPrice"] = 0.0

    for i in range(len(temp)):
        if i == 0:
            temp.loc[i, "NewPrice"] = adaptive_price(10.0, temp.loc[i, "Occupancy"], temp.loc[i, "Capacity"])
        else:
            temp.loc[i, "PreviousPrice"] = temp.loc[i - 1, "NewPrice"]
            temp.loc[i, "NewPrice"] = adaptive_price(temp.loc[i, "PreviousPrice"], temp.loc[i, "Occupancy"], temp.loc[i, "Capacity"])

    results.append(temp)

# Combine all results
df_final = pd.concat(results).reset_index(drop=True)


In [18]:
# Save the selected columns to a CSV file for streaming or downstream processing
# Save only necessary columns to stream
df_final_for_stream = df_final[["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity", "PreviousPrice", "NewPrice"]]
df_final_for_stream.to_csv("parking_stream.csv", index=False)



# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [24]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    PreviousPrice: float
    NewPrice: float
    SystemCodeNumber: str


In [25]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [26]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)
data_with_price = data.with_columns(
    Price = data.NewPrice,
    ParkingLot = data.SystemCodeNumber  # Optional: keeps lot info in output
)



In [33]:
pw.io.jsonlines.write(data_with_price, "Output11.jsonl")
pw.run()

Output()

In [56]:
df_output = pd.read_json("Output11.jsonl", lines=True)
df_output.head(10)


,Timestamp,Occupancy,Capacity,PreviousPrice,NewPrice,SystemCodeNumber,Price,ParkingLot,diff,time
0,2016-10-04 08:25:00,64,577,10.02,10.04,BHMBCCMKT01,10.04,BHMBCCMKT01,1,1751606010846
1,2016-10-04 09:32:00,107,577,10.08,10.15,BHMBCCMKT01,10.15,BHMBCCMKT01,1,1751606010846
2,2016-10-04 08:59:00,80,577,10.04,10.08,BHMBCCMKT01,10.08,BHMBCCMKT01,1,1751606010846
3,2016-10-04 07:59:00,61,577,10.00,10.02,BHMBCCMKT01,10.02,BHMBCCMKT01,1,1751606010846
4,2016-10-04 09:59:00,150,577,10.15,10.29,BHMBCCMKT01,10.29,BHMBCCMKT01,1,1751606010846
5,2016-10-04 10:26:00,177,577,10.29,10.48,BHMBCCMKT01,10.48,BHMBCCMKT01,1,1751606010848
6,2016-10-04 10:59:00,219,577,10.48,10.77,BHMBCCMKT01,10.77,BHMBCCMKT01,1,1751606010848
7,2016-10-04 11:25:00,247,577,10.77,11.14,BHMBCCMKT01,11.14,BHMBCCMKT01,1,1751606010850
8,2016-10-04 11:59:00,259,577,11.14,11.54,BHMBCCMKT01,11.54,BHMBCCMKT01,1,1751606010850
9,2016-10-04 12:29:00,266,577,11.54,11.97,BHMBCCMKT01,11.97,BHMBCCMKT01,1,1751606010852


In [40]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
import pandas as pd

output_notebook()

# Load your output
df_output = pd.read_json("Output11.jsonl", lines=True)
df_output["Timestamp"] = pd.to_datetime(df_output["Timestamp"])

plots = []

# Loop over all parking lots
for lot_id in df_output["SystemCodeNumber"].unique():
    temp = df_output[df_output["SystemCodeNumber"] == lot_id].sort_values("Timestamp")

    p = figure(x_axis_type='datetime',
               title=f"Price Over Time — Lot {lot_id}",
               width=700, height=300)

    p.line(temp["Timestamp"], temp["Price"], line_width=2, color="green")
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price (₹)"

    plots.append(p)

# Show all plots stacked
show(column(*plots))


In [48]:
print(df.columns)
print(df["SystemCodeNumber"].unique())


Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'Timestamp',
       'PreviousPrice', 'NewPrice', 'OccRate', 'QueueLengthNorm',
       'TrafficNorm', 'VehicleTypeWeight'],
      dtype='object')
[]


# Step 2: Making a simple pricing function

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [41]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

NameError: name 'delta_window' is not defined

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()